In [10]:
from nltk.corpus import stopwords
sw = set(stopwords.words('english'))


In [24]:
from gensim.models import KeyedVectors
from gensim.models import Word2Vec
import numpy as np


def restrict_w2v(w2v, restricted_word_set):
    
    new_vectors = []
    new_vocab = {}
    new_index2entity = []

    for i in range(len(w2v.vocab)):
        
        word = w2v.index2entity[i]
        
        if word in restricted_word_set:
            
            vec = w2v.vectors[i]
            vocab = w2v.vocab[word]
            
            vocab.index = len(new_index2entity)
            new_index2entity.append(word)
            new_vocab[word] = vocab
            new_vectors.append(vec)
            
        
    w2v.vocab = new_vocab
    w2v.vectors = np.vstack(new_vectors)
    w2v.index2entity = new_index2entity
    w2v.index2word = new_index2entity
    # w2v.vectors_norm = None
    

model = Word2Vec.load('../word2vec_models/word2vec_bmc_30.model')
word_vectors = model.wv

sel_word = []

for word, vocab in word_vectors.vocab.items():
    
    if word in sw: continue
    
    if ('UMLS_' in word) and (len(word) == 13):
        sel_word.append(word)
    
    if vocab.count >= 200:
        sel_word.append(word)

sel_word = set(sel_word)

restrict_w2v(word_vectors, sel_word)
word_vectors.save('trimmed_wv.kv')



/home/chaozhang/.local/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/home/chaozhang/.local/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL
/home/chaozhang/.local/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migr

In [26]:
word_vectors.most_similar('drop', )

[('UMLS_C0085639', 0.6168836951255798),
 ('reach', 0.6148582696914673),
 ('decrease', 0.5975610613822937),
 ('dip', 0.5654827952384949),
 ('increase', 0.5651909112930298),
 ('reduction', 0.5589075684547424),
 ('return', 0.5532367825508118),
 ('squeeze', 0.5507760047912598),
 ('again', 0.5421923995018005),
 ('turn', 0.5415719151496887)]

In [55]:
import pandas as pd
dic = {}
WM = pd.read_csv('../../WeightMatrix/Dis_Sym_30.csv', index_col=0)
SYMP = WM.columns
for word in sel_word:
    if ('UMLS_' not in word):
        for syn in word_vectors.most_similar(word, topn = 30):
            syn = syn[0]
            if 'UMLS_' in syn:
                syn = syn.strip('UMLS_')
                if syn in SYMP:
                    dic[word] = syn
                    continue


In [60]:
dic['wheeze']

'C0037383'

In [61]:
ser = pd.Series(dic)

In [63]:
df = pd.DataFrame(ser)

In [65]:
df.to_csv('Dictionary.csv', header = False)